In [135]:
import csv
import ast
import json
import statistics

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from datetime import datetime, timedelta

In [136]:
date = "20240203"
date_with_dash = "2024-02-03"
port = "4200"
phone_time = "0203"
time = "1003"
sent_file_name = f"log_{date}_{phone_time}_{port}_client"
received_file_name = f"log_{date}_{time}_{port}_server"
path = f"/media/wmnlab/1DBE-7367/MOXA/{date_with_dash}/QUIC-450sec/sm00/#04/"

In [137]:
sync_file_name = f"/media/wmnlab/1DBE-7367/MOXA/{date_with_dash}/QUIC-450sec/time_sync_sm00.json"
# sync_file = path + "raw/" + sync_file_name
with open(sync_file_name, 'r') as file:
    data = json.load(file)

# Extract values from the dictionary
values = list(data.values())
mean_diff = values[3] * 1000

### Transform to JSON & CSV file
Process the qlog file to json file & csv file.

In [138]:
def QlogToJsonEntry(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()

    # Add commas between lines
    json_str = ",".join(lines)
    # Surround the entire string with square brackets to create a JSON array
    json_str = "[" + json_str + "]"
    # Load the JSON array
    json_entry = json.loads(json_str)
    
    return json_entry

def QlogToJson(json_entry, json_file_path):
    with open(json_file_path, 'w') as json_file:
        json.dump(json_entry, json_file, indent=2)

def JsonToCsv(json_entry, csv_file_path):
     # Open CSV file for writing
    with open(csv_file_path, 'w', newline='') as csv_file:
        # Create a CSV writer
        csv_writer = csv.writer(csv_file)

        # Write header row based on the keys of the second JSON object (assuming at least two objects are present)
        if len(json_entry) >= 2:
            header = list(json_entry[1].keys())
            csv_writer.writerow(header)

            # Write data rows starting from the second object
            for entry in json_entry[1:]:
                csv_writer.writerow(entry.values())

In [139]:
# sender_side_file
sent_raw_path = path + "raw/" + sent_file_name
sent_qlog_file_path = sent_raw_path + ".qlog"
sent_json_file_path = sent_raw_path + ".json"
sent_csv_file_path = sent_raw_path + ".csv"
sent_json_entry = QlogToJsonEntry(sent_qlog_file_path)
QlogToJson(sent_json_entry, sent_json_file_path)
JsonToCsv(sent_json_entry, sent_csv_file_path)

In [140]:
received_raw_path = path + "raw/" + received_file_name
received_qlog_file_path = received_raw_path + ".qlog"
received_json_file_path = received_raw_path + ".json"
received_csv_file_path = received_raw_path + ".csv"
received_json_entry = QlogToJsonEntry(received_qlog_file_path)
QlogToJson(received_json_entry, received_json_file_path)
JsonToCsv(received_json_entry, received_csv_file_path)

In [141]:
sent_df = pd.read_csv(sent_csv_file_path)
received_df = pd.read_csv(received_csv_file_path)

Set time to UMT+8.

In [142]:
def GetStartTime(json_data):
    # unit: ms
    refTime = json_data[0]["trace"]["common_fields"]["reference_time"]
    return refTime

def ProcessTime(df, reference_time):
    # Extract the "time" values from the DataFrame
    original_times = (df['time'].astype(float))

    # Calculate "epoch_time" and convert to timestamps
    epoch_times = (reference_time + original_times)
    timestamps = pd.to_datetime(epoch_times, unit='ms').dt.strftime('%Y-%m-%d %H:%M:%S.%f')

    df['epoch_time'] = epoch_times
    df['timestamp'] = timestamps

    return df

In [143]:
# No matter downlink or uplink, the file time that need to change is client side.
if int(port)%2 == 0: # UL
    clientStartTime = GetStartTime(sent_json_entry)
    print(clientStartTime)
    serverStartTime = GetStartTime(received_json_entry)
    print(serverStartTime)

    senderRefTime = clientStartTime + mean_diff
    rcverRefTime = serverStartTime

else:   # DL
    clientStartTime = GetStartTime(received_json_entry)
    print(clientStartTime)
    serverStartTime = GetStartTime(sent_json_entry)
    print(serverStartTime)
    startTimeDiff = (clientStartTime - serverStartTime) + mean_diff

    senderRefTime = serverStartTime
    rcverRefTime = clientStartTime + mean_diff


1706925796238.9597
1706925786278.4912


In [144]:
sent_df = ProcessTime(sent_df, senderRefTime)
# Add 8 hours to both epoch times and timestamps to match UMT+8
# Also sync time with server
epoch_times_gmt8 = sent_df["epoch_time"] + 8 * 3600 * 1000
sent_df["epoch_time"] = epoch_times_gmt8
timestamps_gmt8 = pd.to_datetime(epoch_times_gmt8, unit='ms').dt.strftime('%Y-%m-%d %H:%M:%S.%f')
sent_df["timestamp"] = timestamps_gmt8

sent_df[-5:]

,time,name,data,epoch_time,timestamp
570476,450082.215349,recovery:loss_timer_updated,{'event_type': 'cancelled'},1.706955e+12,2024-02-03 10:10:36.344799
570477,450082.224932,transport:packet_received,"{'header': {'packet_type': '1RTT', 'packet_num...",1.706955e+12,2024-02-03 10:10:36.344809
570478,450088.328474,transport:packet_sent,"{'header': {'packet_type': '1RTT', 'dcid': '6f...",1.706955e+12,2024-02-03 10:10:36.350912
570479,451055.240974,transport:connection_closed,"{'owner': 'local', 'application_code': 0, 'rea...",1.706955e+12,2024-02-03 10:10:37.317825
570480,451055.336130,transport:packet_sent,"{'header': {'packet_type': '1RTT', 'dcid': '6f...",1.706955e+12,2024-02-03 10:10:37.317920


In [145]:
received_df = ProcessTime(received_df, rcverRefTime)
# if the sender is server, then it is no need to calculate time difference
epoch_times_gmt8 = received_df["epoch_time"] + 8 * 3600 * 1000
received_df["epoch_time"] = epoch_times_gmt8
timestamps_gmt8 = pd.to_datetime(epoch_times_gmt8, unit='ms').dt.strftime('%Y-%m-%d %H:%M:%S.%f')
received_df["timestamp"] = timestamps_gmt8

received_df[-5:]

,time,name,data,epoch_time,timestamp
199637,450060.526519,recovery:metrics_updated,{'bytes_in_flight': 0},1.706955e+12,2024-02-03 10:10:36.339017
199638,450060.527736,recovery:loss_timer_updated,{'event_type': 'cancelled'},1.706955e+12,2024-02-03 10:10:36.339019
199639,450060.528981,transport:packet_received,"{'header': {'packet_type': '1RTT', 'packet_num...",1.706955e+12,2024-02-03 10:10:36.339020
199640,451059.958629,transport:packet_received,"{'header': {'packet_type': '1RTT', 'packet_num...",1.706955e+12,2024-02-03 10:10:37.338449
199641,451059.992832,transport:connection_closed,"{'owner': 'remote', 'application_code': 0, 're...",1.706955e+12,2024-02-03 10:10:37.338484


Parse the data.

In [146]:
# sender side data
metrics_all_rows = sent_df[(sent_df['name'] == 'recovery:metrics_updated') & (sent_df['data'].str.contains("'bytes_in_flight':"))]
metrics_sent_rows = sent_df[(sent_df['name'] == 'recovery:metrics_updated') & (sent_df['data'].str.contains("{'bytes_in_flight':"))]
metrics_ack_rows = sent_df[(sent_df['name'] == 'recovery:metrics_updated') & (sent_df['data'].str.contains("'latest_rtt':"))]
total_sent_rows = sent_df[(sent_df['name'] == 'transport:packet_sent')]
pk_sent_rows = sent_df[(sent_df['name'] == 'transport:packet_sent') & (sent_df['data'].str.contains("'frame_type': 'stream'"))]
rcv_ack_rows = sent_df[(sent_df['name'] == 'transport:packet_received') & (sent_df['data'].str.contains("'frame_type': 'ack'")) & (sent_df['data'].str.contains("'packet_type': '1RTT'"))]
lost_rows = sent_df[sent_df['name'] == 'recovery:packet_lost']

# Get the count of rows
metrics_all_cnt = len(metrics_all_rows)
metrics_c_cnt = len(metrics_sent_rows)
metrics_ack_cnt = len(metrics_ack_rows)
total_sent_cnt = len(total_sent_rows)
pk_sent_cnt = len(pk_sent_rows)
rcv_ack_cnt = len(rcv_ack_rows)
lost_cnt = len(lost_rows)

print("packet_sent: ", pk_sent_cnt, metrics_c_cnt)
print("ack: ", rcv_ack_cnt, metrics_ack_cnt)
print(metrics_all_cnt, metrics_c_cnt, metrics_ack_cnt, pk_sent_cnt, rcv_ack_cnt, lost_cnt)

packet_sent:  149934 149938
ack:  39932 39891
189829 149938 39891 149934 39932 140


In [147]:
pk_rcv_rows = received_df[(received_df['name'] == "transport:packet_received") & (received_df['data'].str.contains("'frame_type': 'stream'"))]
pk_rcv_rows = pk_rcv_rows.reset_index(drop=True)
print(len(pk_rcv_rows))
pk_rcv_rows[:5]

149830


,time,name,data,epoch_time,timestamp
0,31.076779,transport:packet_received,"{'header': {'packet_type': '1RTT', 'dcid': '12...",1.706955e+12,2024-02-03 10:03:06.309567
1,43.031544,transport:packet_received,"{'header': {'packet_type': '1RTT', 'packet_num...",1.706955e+12,2024-02-03 10:03:06.321522
2,43.037217,transport:packet_received,"{'header': {'packet_type': '1RTT', 'packet_num...",1.706955e+12,2024-02-03 10:03:06.321528
3,43.040325,transport:packet_received,"{'header': {'packet_type': '1RTT', 'packet_num...",1.706955e+12,2024-02-03 10:03:06.321531
4,43.042879,transport:packet_received,"{'header': {'packet_type': '1RTT', 'packet_num...",1.706955e+12,2024-02-03 10:03:06.321534


## Deal with sender side data
Concat `transport:packet_sent` & `recovery:metrics_updated`.

In [148]:
metrics_sent_csv_file_path = path + "middle/" + f"sent_metrics_{time}_{port}.csv"
metrics_sent_rows.to_csv(metrics_sent_csv_file_path, index=False)
pk_sent_csv_file_path = path + "middle/" + f"pk_sent_{time}_{port}.csv"
pk_sent_rows.to_csv(pk_sent_csv_file_path, index=False)

In [149]:
def insert(df, idx, new_row):
    df1 = df.iloc[:idx, :]
    df2 = df.iloc[idx:, :]
    df_new = pd.concat([df1, new_row, df2], ignore_index=True)
    return df_new

In [150]:
metrics_sent_rows = metrics_sent_rows.reset_index(drop=True)
pk_sent_rows = pk_sent_rows.reset_index(drop=True)
print(metrics_sent_rows[:5])
print(pk_sent_rows[:5])


        time                      name  \
0  50.444375  recovery:metrics_updated   
1  50.460989  recovery:metrics_updated   
2  50.732552  recovery:metrics_updated   
3  53.410052  recovery:metrics_updated   
4  56.213125  recovery:metrics_updated   

                                               data    epoch_time  \
0   {'bytes_in_flight': 70, 'packets_in_flight': 1}  1.706955e+12   
1   {'bytes_in_flight': 95, 'packets_in_flight': 2}  1.706955e+12   
2  {'bytes_in_flight': 370, 'packets_in_flight': 4}  1.706955e+12   
3  {'bytes_in_flight': 647, 'packets_in_flight': 6}  1.706955e+12   
4  {'bytes_in_flight': 924, 'packets_in_flight': 7}  1.706955e+12   

                    timestamp  
0  2024-02-03 10:03:06.313028  
1  2024-02-03 10:03:06.313045  
2  2024-02-03 10:03:06.313317  
3  2024-02-03 10:03:06.315994  
4  2024-02-03 10:03:06.318797  
        time                   name  \
0  50.717656  transport:packet_sent   
1  53.398489  transport:packet_sent   
2  56.163906  transport

In [151]:
ori_recover_c_len = len(metrics_sent_rows)
for i in range(pk_sent_cnt):
    if(i >= len(metrics_sent_rows)):
        data = metrics_sent_rows.iloc[i-1]['data']
        new_row_data = {'time': [pk_sent_rows.iloc[i]['time']], 'name':['recovery:metrics_updated'], 'data': [data]}
        new_row = pd.DataFrame(new_row_data)
        metrics_sent_rows = pd.concat([metrics_sent_rows, new_row], ignore_index=True)
        continue
    time_diff = metrics_sent_rows.iloc[i]['time'] - pk_sent_rows.iloc[i]['time']
    # print(i, time_diff)
    # time_diff >= 1: not the matching metrics_update
    while time_diff >= 1:
        data = metrics_sent_rows.iloc[i-1]['data']
        new_row_data = {'time': [pk_sent_rows.iloc[i]['time']], 'name':['recovery:metrics_updated'], 'data': [data]}
        new_row = pd.DataFrame(new_row_data)
        # print(new_row)
        metrics_sent_rows = insert(metrics_sent_rows, i, new_row)
        time_diff = metrics_sent_rows.iloc[i]['time'] - pk_sent_rows.iloc[i]['time']
    # time_diff < 0: missing metrics_update
    while time_diff < 0:
        # print(i, time_diff_list)
        metrics_sent_rows.drop(index=metrics_sent_rows.index[i], inplace=True)
        time_diff = metrics_sent_rows.iloc[i]['time'] - pk_sent_rows.iloc[i]['time']

    

# if len(metrics_sent_rows) < pk_sent_cnt:
#     d = pk_sent_cnt - len(metrics_sent_rows)
# data = metrics_sent_rows.iloc[len(metrics_sent_rows)-1]['data']

# for i in range(d):
#     last_row_data = {'time': [pk_sent_rows.iloc[len(metrics_sent_rows)-1]['time']], 'name':['recovery:metrics_updated'], 'data': [data]}
#     new_row_df = pd.DataFrame(last_row_data)
#     metrics_sent_rows = pd.concat([metrics_sent_rows, new_row], ignore_index=True)

print(ori_recover_c_len, len(metrics_sent_rows))


149938 149934


In [152]:
metrics_sent_rows = metrics_sent_rows.reset_index(drop=True)
pk_sent_rows = pk_sent_rows.reset_index(drop=True)
print(len(metrics_sent_rows), len(pk_sent_rows))

# check whether there's still mismatch exist.
time_diff_list = metrics_sent_rows['time'] - pk_sent_rows['time']
mismatch_indices = time_diff_list[(time_diff_list >= 1) | (time_diff_list < 0)].index
if len(mismatch_indices) == 0:
    print("All Matched!")
else:
    print(mismatch_indices)


149934 149934
Index([147628], dtype='int64')


In [153]:
# extract bytes_in_flight & packets_in_flight
metrics_sent_rows['bytes_in_flight'] = None
metrics_sent_rows['packets_in_flight'] = None

# Use ast.literal_eval to safely evaluate the string and extract 'bytes_in_flight' and 'packets_in_flight'
metrics_sent_rows[['bytes_in_flight', 'packets_in_flight']] = metrics_sent_rows['data'].apply(
    lambda x: pd.Series(ast.literal_eval(x)) if isinstance(x, str) else pd.Series([None, None]))

metrics_sent_rows[:5]

,time,name,data,epoch_time,timestamp,bytes_in_flight,packets_in_flight
0,50.732552,recovery:metrics_updated,"{'bytes_in_flight': 370, 'packets_in_flight': 4}",1.706955e+12,2024-02-03 10:03:06.313317,370.0,4.0
1,53.410052,recovery:metrics_updated,"{'bytes_in_flight': 647, 'packets_in_flight': 6}",1.706955e+12,2024-02-03 10:03:06.315994,647.0,6.0
2,56.213125,recovery:metrics_updated,"{'bytes_in_flight': 924, 'packets_in_flight': 7}",1.706955e+12,2024-02-03 10:03:06.318797,924.0,7.0
3,58.655417,recovery:metrics_updated,"{'bytes_in_flight': 1201, 'packets_in_flight': 8}",1.706955e+12,2024-02-03 10:03:06.321240,1201.0,8.0
4,61.359323,recovery:metrics_updated,"{'bytes_in_flight': 1478, 'packets_in_flight': 9}",1.706955e+12,2024-02-03 10:03:06.323943,1478.0,9.0


In [154]:
# Add bytes_in_flight & packets_in_flight to pk_sent_rows
pk_sent_rows['bytes_in_flight'] = metrics_sent_rows['bytes_in_flight']
pk_sent_rows['packets_in_flight'] = metrics_sent_rows['packets_in_flight']

pk_sent_rows[:5]

,time,name,data,epoch_time,timestamp,bytes_in_flight,packets_in_flight
0,50.717656,transport:packet_sent,"{'header': {'packet_type': '1RTT', 'dcid': '12...",1.706955e+12,2024-02-03 10:03:06.313302,370.0,4.0
1,53.398489,transport:packet_sent,"{'header': {'packet_type': '1RTT', 'dcid': '12...",1.706955e+12,2024-02-03 10:03:06.315982,647.0,6.0
2,56.163906,transport:packet_sent,"{'header': {'packet_type': '1RTT', 'dcid': '12...",1.706955e+12,2024-02-03 10:03:06.318748,924.0,7.0
3,58.643594,transport:packet_sent,"{'header': {'packet_type': '1RTT', 'dcid': '12...",1.706955e+12,2024-02-03 10:03:06.321228,1201.0,8.0
4,61.329271,transport:packet_sent,"{'header': {'packet_type': '1RTT', 'dcid': '12...",1.706955e+12,2024-02-03 10:03:06.323913,1478.0,9.0


Concat `transport:packet_received` & `recovery:metrics_updated`.

In [155]:
metrics_ack_csv_file_path = path + "middle/" + f"ack_metrics_{time}_{port}.csv" 
metrics_ack_rows.to_csv(metrics_ack_csv_file_path, index=False)
rcv_ack_csv_file_path = path + "middle/" + f"rcv_ack_{time}_{port}.csv"
rcv_ack_rows.to_csv(rcv_ack_csv_file_path, index=False)

In [156]:
metrics_ack_rows = metrics_ack_rows.reset_index(drop=True)
rcv_ack_rows = rcv_ack_rows.reset_index(drop=True)
initial_ack_metrics = metrics_ack_rows.iloc[[0]]
metrics_ack_rows.drop(index=metrics_ack_rows.index[0], inplace=True)
metrics_ack_rows = metrics_ack_rows.reset_index(drop=True)

In [157]:
metrics_ack_rows[:3]

,time,name,data,epoch_time,timestamp
0,48.101458,recovery:metrics_updated,"{'min_rtt': 46.58026, 'smoothed_rtt': 46.58026...",1.706955e+12,2024-02-03 10:03:06.310685
1,75.801771,recovery:metrics_updated,"{'min_rtt': 25.38224, 'smoothed_rtt': 43.93, '...",1.706955e+12,2024-02-03 10:03:06.338386
2,87.301250,recovery:metrics_updated,"{'smoothed_rtt': 41.669, 'latest_rtt': 25.8426...",1.706955e+12,2024-02-03 10:03:06.349885


In [158]:
rcv_ack_rows[:3]

,time,name,data,epoch_time,timestamp
0,75.932969,transport:packet_received,"{'header': {'packet_type': '1RTT', 'packet_num...",1.706955e+12,2024-02-03 10:03:06.338517
1,87.312135,transport:packet_received,"{'header': {'packet_type': '1RTT', 'packet_num...",1.706955e+12,2024-02-03 10:03:06.349896
2,100.721771,transport:packet_received,"{'header': {'packet_type': '1RTT', 'packet_num...",1.706955e+12,2024-02-03 10:03:06.363306


In [159]:
print(len(metrics_ack_rows), len(rcv_ack_rows))
for i in range(rcv_ack_cnt):
    if(i >= len(metrics_ack_rows)):
        data = metrics_ack_rows.iloc[i-1]['data']
        new_row_data = {'time': [rcv_ack_rows.iloc[i]['time']], 'name':['recovery:metrics_updated'], 'data': [data]}
        new_row = pd.DataFrame(new_row_data)
        metrics_ack_rows = pd.concat([metrics_ack_rows, new_row], ignore_index=True)
        continue
    time_diff = metrics_ack_rows.iloc[i]['time'] - rcv_ack_rows.iloc[i]['time']
    # time_diff >= 1: not the matching metrics_update
    while time_diff > 0:
        # print("> 0:", i, time_diff)
        if i == 0:
            data = initial_ack_metrics.iloc[0]['data']
        else:
            data = metrics_ack_rows.iloc[i-1]['data']
        new_row_data = {'time': [rcv_ack_rows.iloc[i]['time']], 'name':['recovery:metrics_updated'], 'data': [data]}
        new_row = pd.DataFrame(new_row_data)
        metrics_ack_rows = insert(metrics_ack_rows, i, new_row)
        time_diff = metrics_ack_rows.iloc[i]['time'] - rcv_ack_rows.iloc[i]['time']
    # time_diff < 0: missing metrics_update
    while time_diff <= -1:
        # print("<= -1:", i, time_diff)
        metrics_ack_rows.drop(index=metrics_ack_rows.index[i], inplace=True)
        time_diff = metrics_ack_rows.iloc[i]['time'] - rcv_ack_rows.iloc[i]['time']
print(len(metrics_ack_rows), len(rcv_ack_rows))

39890 39932
39932 39932


In [160]:
metrics_ack_rows = metrics_ack_rows.reset_index(drop=True)
rcv_ack_rows = rcv_ack_rows.reset_index(drop=True)

# check whether there's still mismatch exist.
time_diff_list = metrics_ack_rows['time'] - rcv_ack_rows['time']
mismatch_indices = time_diff_list[(time_diff_list <= -1) | (time_diff_list > 0)].index
if len(mismatch_indices) == 0:
    print("All Matched!")
else:
    print(mismatch_indices)

Index([  792,  4703,  6706,  7660,  8227,  9579,  9726, 11250, 13020, 14572,
       14590, 15563, 16972, 18238, 19313, 20389, 20411, 21854, 22506, 23010,
       23546, 24204, 24456, 25096, 25347, 25381, 25652, 26088, 26361, 26397,
       26928, 27605, 28122, 29016, 29196, 30183, 30332, 30497, 31946, 31951,
       32305, 32410, 32490, 33322, 33385, 33442, 33782, 34783, 36473, 36978,
       37105, 37252, 37458, 37497, 37805, 38942, 38981, 39094, 39916],
      dtype='int64')


In [161]:
ack_json_list = []
## Add the initial_ack_metrics for temporary
print(initial_ack_metrics)
metrics_ack_rows = pd.concat([initial_ack_metrics, metrics_ack_rows], axis=0).reset_index(drop=True)
for i in range(len(metrics_ack_rows)):
    s = metrics_ack_rows.iloc[i]['data'].replace("\'", "\"")
    json_object = json.loads(s)
    ack_json_list.append(json_object)

metrics_ack_df = pd.DataFrame(ack_json_list)
# Fill missing values in each row with the previous row's values
metrics_ack_df = metrics_ack_df.ffill(axis=0)

## drop initial_ack_metrics
metrics_ack_rows.drop(index=metrics_ack_rows.index[0], inplace=True)
metrics_ack_rows = metrics_ack_rows.reset_index(drop=True)
metrics_ack_df.drop(index=metrics_ack_df.index[0], inplace=True)
metrics_ack_df = metrics_ack_df.reset_index(drop=True)
metrics_ack_df[:5]

       time                      name  \
0  1.237396  recovery:metrics_updated   

                                                data    epoch_time  \
0  {'min_rtt': 0, 'smoothed_rtt': 0, 'latest_rtt'...  1.706955e+12   

                    timestamp  
0  2024-02-03 10:03:06.263821  


,min_rtt,smoothed_rtt,latest_rtt,rtt_variance,congestion_window,bytes_in_flight,packets_in_flight
0,25.38224,43.930,25.382240,22.767,40064.0,2838,13.0
1,25.38224,41.669,25.842635,21.597,40064.0,2501,9.0
2,25.38224,40.813,34.823010,17.909,40064.0,3332,12.0
3,25.38224,39.510,30.396229,16.035,40064.0,3047,11.0
4,25.38224,38.649,32.625334,13.747,40064.0,3047,11.0


In [162]:
metrics_ack_rows = pd.concat([metrics_ack_rows, metrics_ack_df], axis=1).reset_index(drop=True)
# since we have parse out all the information in data, we can drop the data cl=olumn
metrics_ack_rows = metrics_ack_rows.drop(columns=['data'])
metrics_ack_rows[:5]

,time,name,epoch_time,timestamp,min_rtt,smoothed_rtt,latest_rtt,rtt_variance,congestion_window,bytes_in_flight,packets_in_flight
0,75.801771,recovery:metrics_updated,1.706955e+12,2024-02-03 10:03:06.338386,25.38224,43.930,25.382240,22.767,40064.0,2838,13.0
1,87.301250,recovery:metrics_updated,1.706955e+12,2024-02-03 10:03:06.349885,25.38224,41.669,25.842635,21.597,40064.0,2501,9.0
2,100.712344,recovery:metrics_updated,1.706955e+12,2024-02-03 10:03:06.363296,25.38224,40.813,34.823010,17.909,40064.0,3332,12.0
3,112.555260,recovery:metrics_updated,1.706955e+12,2024-02-03 10:03:06.375139,25.38224,39.510,30.396229,16.035,40064.0,3047,11.0
4,126.067135,recovery:metrics_updated,1.706955e+12,2024-02-03 10:03:06.388651,25.38224,38.649,32.625334,13.747,40064.0,3047,11.0


In [163]:
# Check whehter the length is equal before concating metrics into rcv_ack_rows
print(len(rcv_ack_rows), len(metrics_ack_df))

39932 39932


In [164]:
rcv_ack_rows = pd.concat([rcv_ack_rows, metrics_ack_df], axis=1)
rcv_ack_rows = rcv_ack_rows.reset_index(drop=True)

print(len(rcv_ack_rows), len(metrics_ack_df))
rcv_ack_rows[-5:]

39932 39932


,time,name,data,epoch_time,timestamp,min_rtt,smoothed_rtt,latest_rtt,rtt_variance,congestion_window,bytes_in_flight,packets_in_flight
39927,450038.976287,transport:packet_received,"{'header': {'packet_type': '1RTT', 'packet_num...",1.706955e+12,2024-02-03 10:10:36.301560,13.571041,22.394,20.598770,1.845,9997.0,1674,6.0
39928,450051.235662,transport:packet_received,"{'header': {'packet_type': '1RTT', 'packet_num...",1.706955e+12,2024-02-03 10:10:36.313820,13.571041,22.159,20.514364,1.853,9997.0,1953,7.0
39929,450063.377901,transport:packet_received,"{'header': {'packet_type': '1RTT', 'packet_num...",1.706955e+12,2024-02-03 10:10:36.325962,13.571041,22.409,24.160021,1.890,9997.0,1116,4.0
39930,450063.419412,transport:packet_received,"{'header': {'packet_type': '1RTT', 'packet_num...",1.706955e+12,2024-02-03 10:10:36.326003,13.571041,21.928,18.564916,2.378,9997.0,558,2.0
39931,450082.224932,transport:packet_received,"{'header': {'packet_type': '1RTT', 'packet_num...",1.706955e+12,2024-02-03 10:10:36.344809,13.571041,23.066,31.037833,4.060,9997.0,0,2.0


Mapping the ACK ranges

In [165]:
acked_ranges_series = rcv_ack_rows['data']
acked_ranges_list = []
for i in range(len(acked_ranges_series)):
    s = acked_ranges_series.iloc[i]
    data_dict = json.loads(s.replace("\'", "\""))
    # Extract 'acked_ranges' from all frames
    acked_ranges = [range_entry for frame in data_dict['frames'] if 'acked_ranges' in frame for range_entry in frame['acked_ranges']]
    acked_ranges_list.append(acked_ranges)

acked_ranges_df = pd.DataFrame({"acked_ranges": acked_ranges_list})
acked_ranges_df[:5]

,acked_ranges
0,[[0]]
1,"[[0, 6]]"
2,"[[0, 8]]"
3,"[[1, 14]]"
4,"[[1, 18]]"


In [166]:
rcv_ack_rows = pd.concat([rcv_ack_rows, acked_ranges_df], axis=1)
rcv_ack_rows = rcv_ack_rows.reset_index(drop=True)

rcv_ack_rows[:5]

,time,name,data,epoch_time,timestamp,min_rtt,smoothed_rtt,latest_rtt,rtt_variance,congestion_window,bytes_in_flight,packets_in_flight,acked_ranges
0,75.932969,transport:packet_received,"{'header': {'packet_type': '1RTT', 'packet_num...",1.706955e+12,2024-02-03 10:03:06.338517,25.38224,43.930,25.382240,22.767,40064.0,2838,13.0,[[0]]
1,87.312135,transport:packet_received,"{'header': {'packet_type': '1RTT', 'packet_num...",1.706955e+12,2024-02-03 10:03:06.349896,25.38224,41.669,25.842635,21.597,40064.0,2501,9.0,"[[0, 6]]"
2,100.721771,transport:packet_received,"{'header': {'packet_type': '1RTT', 'packet_num...",1.706955e+12,2024-02-03 10:03:06.363306,25.38224,40.813,34.823010,17.909,40064.0,3332,12.0,"[[0, 8]]"
3,112.565781,transport:packet_received,"{'header': {'packet_type': '1RTT', 'packet_num...",1.706955e+12,2024-02-03 10:03:06.375150,25.38224,39.510,30.396229,16.035,40064.0,3047,11.0,"[[1, 14]]"
4,126.080260,transport:packet_received,"{'header': {'packet_type': '1RTT', 'packet_num...",1.706955e+12,2024-02-03 10:03:06.388664,25.38224,38.649,32.625334,13.747,40064.0,3047,11.0,"[[1, 18]]"


In [167]:
# parse out the packet_number & offset & length
pk_sent_series =  pk_sent_rows['data']
pk_num_list = []
offset_list = []
length_list = []
for i in range(len(pk_sent_series)):
    s = pk_sent_series.iloc[i]
    data_dict = json.loads(s.replace("\'", "\""))
    packet_number = data_dict['header']['packet_number']
    # Initialize offset to None in case 'frame_type': 'stream' is not found
    offset = None
    # Iterate through frames to find 'offset' for 'frame_type': 'stream'
    for frame in data_dict.get('frames', []):
        if frame.get('frame_type') == 'stream':
            offset = frame.get('offset')
            length = frame.get('length')
            break  # Stop iterating once 'offset' is found
    
    pk_num_list.append(packet_number)
    offset_list.append(offset)
    length_list.append(length)

pk_num_df = pd.DataFrame({"packet_number": pk_num_list, "offset": offset_list, "length": length_list})
pk_num_df[:5]

,packet_number,offset,length
0,1,0,250
1,3,250,250
2,4,500,250
3,5,750,250
4,6,1000,250


In [168]:
pk_sent_rows = pd.concat([pk_sent_rows, pk_num_df], axis=1)
pk_sent_rows = pk_sent_rows.reset_index(drop=True)

pk_sent_rows[:5]

,time,name,data,epoch_time,timestamp,bytes_in_flight,packets_in_flight,packet_number,offset,length
0,50.717656,transport:packet_sent,"{'header': {'packet_type': '1RTT', 'dcid': '12...",1.706955e+12,2024-02-03 10:03:06.313302,370.0,4.0,1,0,250
1,53.398489,transport:packet_sent,"{'header': {'packet_type': '1RTT', 'dcid': '12...",1.706955e+12,2024-02-03 10:03:06.315982,647.0,6.0,3,250,250
2,56.163906,transport:packet_sent,"{'header': {'packet_type': '1RTT', 'dcid': '12...",1.706955e+12,2024-02-03 10:03:06.318748,924.0,7.0,4,500,250
3,58.643594,transport:packet_sent,"{'header': {'packet_type': '1RTT', 'dcid': '12...",1.706955e+12,2024-02-03 10:03:06.321228,1201.0,8.0,5,750,250
4,61.329271,transport:packet_sent,"{'header': {'packet_type': '1RTT', 'dcid': '12...",1.706955e+12,2024-02-03 10:03:06.323913,1478.0,9.0,6,1000,250


In [169]:
pk_sent_rows['smoothed_rtt'] = np.nan
pk_sent_rows['latest_rtt'] = np.nan
pk_sent_rows['rtt_variance'] = np.nan
pk_sent_rows['congestion_window'] = np.nan

pk_sent_rows[:5]

,time,name,data,epoch_time,timestamp,bytes_in_flight,packets_in_flight,packet_number,offset,length,smoothed_rtt,latest_rtt,rtt_variance,congestion_window
0,50.717656,transport:packet_sent,"{'header': {'packet_type': '1RTT', 'dcid': '12...",1.706955e+12,2024-02-03 10:03:06.313302,370.0,4.0,1,0,250,NaN,NaN,NaN,NaN
1,53.398489,transport:packet_sent,"{'header': {'packet_type': '1RTT', 'dcid': '12...",1.706955e+12,2024-02-03 10:03:06.315982,647.0,6.0,3,250,250,NaN,NaN,NaN,NaN
2,56.163906,transport:packet_sent,"{'header': {'packet_type': '1RTT', 'dcid': '12...",1.706955e+12,2024-02-03 10:03:06.318748,924.0,7.0,4,500,250,NaN,NaN,NaN,NaN
3,58.643594,transport:packet_sent,"{'header': {'packet_type': '1RTT', 'dcid': '12...",1.706955e+12,2024-02-03 10:03:06.321228,1201.0,8.0,5,750,250,NaN,NaN,NaN,NaN
4,61.329271,transport:packet_sent,"{'header': {'packet_type': '1RTT', 'dcid': '12...",1.706955e+12,2024-02-03 10:03:06.323913,1478.0,9.0,6,1000,250,NaN,NaN,NaN,NaN


In [170]:
def update_pk_sent_rows(row):
    acked_ranges = row['acked_ranges']
    smoothed_rtt = row['smoothed_rtt']
    latest_rtt = row['latest_rtt']
    rtt_variance = row['rtt_variance']
    congestion_window = row['congestion_window']

    for ack_range in acked_ranges:
        start_packet, end_packet = ack_range[0], ack_range[-1]
        existing_packets = set(pk_sent_rows['packet_number'])
        packet_numbers_to_update = set(range(start_packet, end_packet + 1)).intersection(existing_packets)

        mask = pk_sent_rows['packet_number'].isin(packet_numbers_to_update)
        pk_sent_rows.loc[mask, 'smoothed_rtt'] = pk_sent_rows.loc[mask, 'smoothed_rtt'].fillna(smoothed_rtt)
        pk_sent_rows.loc[mask, 'latest_rtt'] = pk_sent_rows.loc[mask, 'latest_rtt'].fillna(latest_rtt)
        pk_sent_rows.loc[mask, 'congestion_window'] = pk_sent_rows.loc[mask, 'congestion_window'].fillna(congestion_window)
        pk_sent_rows.loc[mask, 'rtt_variance'] = pk_sent_rows.loc[mask, 'rtt_variance'].fillna(rtt_variance)

# Apply the custom update function to each row in rcv_ack_rows
rcv_ack_rows.apply(update_pk_sent_rows, axis=1)

# Display the updated pk_sent_rows
pk_sent_rows[:5]

,time,name,data,epoch_time,timestamp,bytes_in_flight,packets_in_flight,packet_number,offset,length,smoothed_rtt,latest_rtt,rtt_variance,congestion_window
0,50.717656,transport:packet_sent,"{'header': {'packet_type': '1RTT', 'dcid': '12...",1.706955e+12,2024-02-03 10:03:06.313302,370.0,4.0,1,0,250,41.669,25.842635,21.597,40064.0
1,53.398489,transport:packet_sent,"{'header': {'packet_type': '1RTT', 'dcid': '12...",1.706955e+12,2024-02-03 10:03:06.315982,647.0,6.0,3,250,250,41.669,25.842635,21.597,40064.0
2,56.163906,transport:packet_sent,"{'header': {'packet_type': '1RTT', 'dcid': '12...",1.706955e+12,2024-02-03 10:03:06.318748,924.0,7.0,4,500,250,41.669,25.842635,21.597,40064.0
3,58.643594,transport:packet_sent,"{'header': {'packet_type': '1RTT', 'dcid': '12...",1.706955e+12,2024-02-03 10:03:06.321228,1201.0,8.0,5,750,250,41.669,25.842635,21.597,40064.0
4,61.329271,transport:packet_sent,"{'header': {'packet_type': '1RTT', 'dcid': '12...",1.706955e+12,2024-02-03 10:03:06.323913,1478.0,9.0,6,1000,250,41.669,25.842635,21.597,40064.0


Identify lost packets

In [171]:
# Use ast.literal_eval to safely evaluate the string and extract 'packet_number'
lost_rows['packet_number'] = lost_rows['data'].apply(lambda x: ast.literal_eval(x)['header']['packet_number'] if isinstance(x, str) else None)
lost_rows['trigger'] = lost_rows['data'].apply(lambda x: ast.literal_eval(x)['trigger'] if isinstance(x, str) else None)
lost_rows[:5]

/tmp/ipykernel_193643/2583690328.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  lost_rows['packet_number'] = lost_rows['data'].apply(lambda x: ast.literal_eval(x)['header']['packet_number'] if isinstance(x, str) else None)
/tmp/ipykernel_193643/2583690328.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  lost_rows['trigger'] = lost_rows['data'].apply(lambda x: ast.literal_eval(x)['trigger'] if isinstance(x, str) else None)


,time,name,data,epoch_time,timestamp,packet_number,trigger
1518,1184.618801,recovery:packet_lost,"{'header': {'packet_type': '1RTT', 'packet_num...",1.706955e+12,2024-02-03 10:03:07.447203,358,reordering_threshold
1520,1184.637239,recovery:packet_lost,"{'header': {'packet_type': '1RTT', 'packet_num...",1.706955e+12,2024-02-03 10:03:07.447221,359,reordering_threshold
1521,1184.640937,recovery:packet_lost,"{'header': {'packet_type': '1RTT', 'packet_num...",1.706955e+12,2024-02-03 10:03:07.447225,360,reordering_threshold
1525,1184.688333,recovery:packet_lost,"{'header': {'packet_type': '1RTT', 'packet_num...",1.706955e+12,2024-02-03 10:03:07.447272,361,time_threshold
2902,2296.648905,recovery:packet_lost,"{'header': {'packet_type': '1RTT', 'packet_num...",1.706955e+12,2024-02-03 10:03:08.559233,735,time_threshold


In [172]:
lost_pk_csv_file_path = path + "middle/" + f"lost_pk_{time}_{port}.csv"
lost_rows.to_csv(lost_pk_csv_file_path, index=False)

In [173]:
## set to True if the packet is lost
pk_sent_rows['packet_lost'] = False

# Iterate through rows and set 'packet_lost' to True where 'packet_number' values match
for _, lost_row in lost_rows.iterrows():
    packet_number = lost_row['packet_number']
    
    # Check if 'packet_number' exists in pk_sent_rows
    if packet_number in pk_sent_rows['packet_number'].values:
        pk_sent_rows.loc[pk_sent_rows['packet_number'] == packet_number, 'packet_lost'] = True

pk_sent_rows[19340:19345]

,time,name,data,epoch_time,timestamp,bytes_in_flight,packets_in_flight,packet_number,offset,length,smoothed_rtt,latest_rtt,rtt_variance,congestion_window,packet_lost
19340,55577.259145,transport:packet_sent,"{'header': {'packet_type': '1RTT', 'dcid': 'a0...",1.706955e+12,2024-02-03 10:04:01.839843,2790.0,10.0,19361,4838750,250,22.636,19.095990,2.948,15220.0,False
19341,55579.842427,transport:packet_sent,"{'header': {'packet_type': '1RTT', 'dcid': 'a0...",1.706955e+12,2024-02-03 10:04:01.842426,3069.0,11.0,19362,4839000,250,22.636,19.095990,2.948,15220.0,False
19342,55582.442114,transport:packet_sent,"{'header': {'packet_type': '1RTT', 'dcid': 'a0...",1.706955e+12,2024-02-03 10:04:01.845026,3348.0,12.0,19363,4839250,250,22.636,19.095990,2.948,15220.0,False
19343,55585.231072,transport:packet_sent,"{'header': {'packet_type': '1RTT', 'dcid': 'a0...",1.706955e+12,2024-02-03 10:04:01.847815,3627.0,13.0,19364,4839500,250,22.636,19.095990,2.948,15220.0,False
19344,55587.497010,transport:packet_sent,"{'header': {'packet_type': '1RTT', 'dcid': 'a0...",1.706955e+12,2024-02-03 10:04:01.850081,2790.0,10.0,19365,4839750,250,22.426,20.958375,2.630,15220.0,False


In [174]:
cols = ['time', 'epoch_time', 'timestamp', 'name', 'packet_number', 'offset', 'length', 'bytes_in_flight', 'packets_in_flight', 'smoothed_rtt', 'latest_rtt', 'rtt_variance', 'congestion_window', 'packet_lost', 'data']
processed_df = pk_sent_rows[cols]
processed_df[:5]

,time,epoch_time,timestamp,name,packet_number,offset,length,bytes_in_flight,packets_in_flight,smoothed_rtt,latest_rtt,rtt_variance,congestion_window,packet_lost,data
0,50.717656,1.706955e+12,2024-02-03 10:03:06.313302,transport:packet_sent,1,0,250,370.0,4.0,41.669,25.842635,21.597,40064.0,False,"{'header': {'packet_type': '1RTT', 'dcid': '12..."
1,53.398489,1.706955e+12,2024-02-03 10:03:06.315982,transport:packet_sent,3,250,250,647.0,6.0,41.669,25.842635,21.597,40064.0,False,"{'header': {'packet_type': '1RTT', 'dcid': '12..."
2,56.163906,1.706955e+12,2024-02-03 10:03:06.318748,transport:packet_sent,4,500,250,924.0,7.0,41.669,25.842635,21.597,40064.0,False,"{'header': {'packet_type': '1RTT', 'dcid': '12..."
3,58.643594,1.706955e+12,2024-02-03 10:03:06.321228,transport:packet_sent,5,750,250,1201.0,8.0,41.669,25.842635,21.597,40064.0,False,"{'header': {'packet_type': '1RTT', 'dcid': '12..."
4,61.329271,1.706955e+12,2024-02-03 10:03:06.323913,transport:packet_sent,6,1000,250,1478.0,9.0,41.669,25.842635,21.597,40064.0,False,"{'header': {'packet_type': '1RTT', 'dcid': '12..."


In [175]:
csv_file_path = path + "data/" + f"processed_sent_{time}_{port}.csv"
processed_df.to_csv(csv_file_path, sep='@', index=False)

In [176]:
weird_length_list =[]
for i in range(len(processed_df)):
    if processed_df.iloc[i]['length'] != 250:
        weird_length_list.append(processed_df.iloc[i])

weird_length_df = pd.DataFrame(weird_length_list)

In [177]:
print(len(weird_length_df))
weird_length_df[:5]

130


,time,epoch_time,timestamp,name,packet_number,offset,length,bytes_in_flight,packets_in_flight,smoothed_rtt,latest_rtt,rtt_variance,congestion_window,packet_lost,data
404,1193.491822,1.706955e+12,2024-02-03 10:03:07.456076,transport:packet_sent,411,100000,500,7783.0,27.0,44.564,33.623792,17.696,28044.0,False,"{'header': {'packet_type': '1RTT', 'dcid': '12..."
2250,6582.168227,1.706955e+12,2024-02-03 10:03:12.844752,transport:packet_sent,2259,560000,1250,7138.0,23.0,92.271,34.717365,61.970,12496.0,False,"{'header': {'packet_type': '1RTT', 'dcid': '12..."
2260,6629.896977,1.706955e+12,2024-02-03 10:03:12.892481,transport:packet_sent,2269,563500,1250,10370.0,31.0,92.271,34.717365,61.970,12496.0,False,"{'header': {'packet_type': '1RTT', 'dcid': '12..."
2261,6647.536299,1.706955e+12,2024-02-03 10:03:12.910120,transport:packet_sent,2270,564750,1250,9975.0,25.0,92.271,34.717365,61.970,12496.0,False,"{'header': {'packet_type': '1RTT', 'dcid': '12..."
2262,6652.487862,1.706955e+12,2024-02-03 10:03:12.915072,transport:packet_sent,2271,566000,500,9667.0,23.0,92.271,34.717365,61.970,12496.0,False,"{'header': {'packet_type': '1RTT', 'dcid': '12..."


In [178]:
# Sum up the 'length' column
total_length = processed_df['length'].sum()

print(f'Total Length: {total_length}')

Total Length: 37562750


## Receiver side data

In [179]:
pk_rcv_df = pk_rcv_rows.reset_index(drop=True)
pk_rcv_df[:5]

,time,name,data,epoch_time,timestamp
0,31.076779,transport:packet_received,"{'header': {'packet_type': '1RTT', 'dcid': '12...",1.706955e+12,2024-02-03 10:03:06.309567
1,43.031544,transport:packet_received,"{'header': {'packet_type': '1RTT', 'packet_num...",1.706955e+12,2024-02-03 10:03:06.321522
2,43.037217,transport:packet_received,"{'header': {'packet_type': '1RTT', 'packet_num...",1.706955e+12,2024-02-03 10:03:06.321528
3,43.040325,transport:packet_received,"{'header': {'packet_type': '1RTT', 'packet_num...",1.706955e+12,2024-02-03 10:03:06.321531
4,43.042879,transport:packet_received,"{'header': {'packet_type': '1RTT', 'packet_num...",1.706955e+12,2024-02-03 10:03:06.321534


In [180]:
pk_rcv_series =  pk_rcv_df['data']
pk_rcv_num_list = []
offset_rcv_list = []
length_rcv_list = []
for i in range(len(pk_rcv_series)):
    s = pk_rcv_series.iloc[i]
    data_dict = json.loads(s.replace("\'", "\""))
    packet_number = data_dict['header']['packet_number']
    # Initialize offset to None in case 'frame_type': 'stream' is not found
    offset = None
    # Iterate through frames to find 'offset' for 'frame_type': 'stream'
    for frame in data_dict.get('frames', []):
        if frame.get('frame_type') == 'stream':
            offset = frame.get('offset')
            length = frame.get('length')
            break  # Stop iterating once 'offset' is found
    
    pk_rcv_num_list.append(packet_number)
    offset_rcv_list.append(offset)
    length_rcv_list.append(length)

pk_rcv_df['packet_number'] = pk_rcv_num_list
pk_rcv_df['offset'] = offset_rcv_list
pk_rcv_df['length'] = length_rcv_list

pk_rcv_df[:5]

,time,name,data,epoch_time,timestamp,packet_number,offset,length
0,31.076779,transport:packet_received,"{'header': {'packet_type': '1RTT', 'dcid': '12...",1.706955e+12,2024-02-03 10:03:06.309567,1,0,250
1,43.031544,transport:packet_received,"{'header': {'packet_type': '1RTT', 'packet_num...",1.706955e+12,2024-02-03 10:03:06.321522,3,250,250
2,43.037217,transport:packet_received,"{'header': {'packet_type': '1RTT', 'packet_num...",1.706955e+12,2024-02-03 10:03:06.321528,4,500,250
3,43.040325,transport:packet_received,"{'header': {'packet_type': '1RTT', 'packet_num...",1.706955e+12,2024-02-03 10:03:06.321531,5,750,250
4,43.042879,transport:packet_received,"{'header': {'packet_type': '1RTT', 'packet_num...",1.706955e+12,2024-02-03 10:03:06.321534,6,1000,250


In [181]:
cols = ['time', 'epoch_time', 'timestamp', 'name', 'packet_number', 'offset', 'length', 'data']
processed_rcv_df = pk_rcv_df[cols]
processed_rcv_df[:5]


,time,epoch_time,timestamp,name,packet_number,offset,length,data
0,31.076779,1.706955e+12,2024-02-03 10:03:06.309567,transport:packet_received,1,0,250,"{'header': {'packet_type': '1RTT', 'dcid': '12..."
1,43.031544,1.706955e+12,2024-02-03 10:03:06.321522,transport:packet_received,3,250,250,"{'header': {'packet_type': '1RTT', 'packet_num..."
2,43.037217,1.706955e+12,2024-02-03 10:03:06.321528,transport:packet_received,4,500,250,"{'header': {'packet_type': '1RTT', 'packet_num..."
3,43.040325,1.706955e+12,2024-02-03 10:03:06.321531,transport:packet_received,5,750,250,"{'header': {'packet_type': '1RTT', 'packet_num..."
4,43.042879,1.706955e+12,2024-02-03 10:03:06.321534,transport:packet_received,6,1000,250,"{'header': {'packet_type': '1RTT', 'packet_num..."


In [182]:
csv_file_path = path + "data/" + f"processed_rcv_{time}_{port}.csv"
processed_rcv_df.to_csv(csv_file_path, sep='@')

In [ ]:
# weird_length_list =[]
# for i in range(len(pk_rcv_df)):
#     if pk_rcv_df.iloc[i]['length'] != 250:
#         weird_length_list.append(pk_rcv_df.iloc[i])

# weird_length_df = pd.DataFrame(weird_length_list)

In [ ]:
# print(len(weird_length_df))
# weird_length_df[:5]